In [1]:
#hocam en aşşağıda eror veriyor tekli çözünce bazı algoritmalar çalışıyor ama fonksiyonun içinde topluca çalıştırınca çalışmıyor bakarmısınız
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
scaler = StandardScaler()
import pandas_profiling
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from imblearn.metrics import geometric_mean_score as geo
from imblearn.metrics import make_index_balanced_accuracy as iba
from sklearn.metrics import roc_curve, auc
from imblearn.metrics import geometric_mean_score, make_index_balanced_accuracy, classification_report_imbalanced
from pycaret.classification import *

In [2]:
df1=pd.read_csv('event_type.csv')

In [3]:
df2=pd.read_csv('log_feature.csv')

In [4]:
df3=pd.read_csv('resource_type.csv')

In [5]:
df4=pd.read_csv('severity_type.csv')

In [6]:
df5=pd.read_csv('train.csv')

In [7]:
merge1=df5.merge(df4, how="left", left_on='id', right_on='id')
merge2=merge1.merge(df3, how="left", left_on='id', right_on='id')
merge3=merge2.merge(df2, how="left", left_on='id', right_on='id')
df=merge3.merge(df1, how="left", left_on='id', right_on='id')

In [8]:
df.drop_duplicates(subset='id',keep='first',inplace=True)

In [9]:
#pandas_profiling.ProfileReport(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7381 entries, 0 to 61836
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              7381 non-null   int64 
 1   location        7381 non-null   object
 2   fault_severity  7381 non-null   int64 
 3   severity_type   7381 non-null   object
 4   resource_type   7381 non-null   object
 5   log_feature     7381 non-null   object
 6   volume          7381 non-null   int64 
 7   event_type      7381 non-null   object
dtypes: int64(3), object(5)
memory usage: 519.0+ KB


In [11]:
df

,id,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 34
4,9320,location 91,0,severity_type 2,resource_type 2,feature 315,200,event_type 34
8,14394,location 152,1,severity_type 2,resource_type 2,feature 221,1,event_type 35
12,8218,location 931,1,severity_type 1,resource_type 8,feature 80,9,event_type 15
18,14804,location 120,0,severity_type 1,resource_type 2,feature 134,1,event_type 34
...,...,...,...,...,...,...,...,...
61824,870,location 167,0,severity_type 2,resource_type 2,feature 232,1,event_type 34
61828,18068,location 106,0,severity_type 1,resource_type 2,feature 219,1,event_type 20
61830,14111,location 1086,2,severity_type 2,resource_type 8,feature 82,21,event_type 15
61832,15189,location 7,0,severity_type 1,resource_type 8,feature 191,3,event_type 11


In [12]:
df['location']=df['location'].str.extract('(\d+)')
df['severity_type']=df['severity_type'].str.extract('(\d+)')
df['resource_type']=df['resource_type'].str.extract('(\d+)')
df['log_feature']=df['log_feature'].str.extract('(\d+)')
df['event_type']=df['event_type'].str.extract('(\d+)')

In [13]:
df['location']=df['location'].astype('category')
df['severity_type']=df['severity_type'].astype('category')
df['resource_type']=df['resource_type'].astype('category')
df['log_feature']=df['log_feature'].astype('category')
df['event_type']=df['event_type'].astype('category')
df['fault_severity']=df['fault_severity'].astype('category')

In [14]:
df.drop(['id'],axis=1)

,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,118,1,2,2,312,19,34
4,91,0,2,2,315,200,34
8,152,1,2,2,221,1,35
12,931,1,1,8,80,9,15
18,120,0,1,2,134,1,34
...,...,...,...,...,...,...,...
61824,167,0,2,2,232,1,34
61828,106,0,1,2,219,1,20
61830,1086,2,2,8,82,21,15
61832,7,0,1,8,191,3,11


In [15]:
x=df[['location','severity_type','resource_type','log_feature','event_type','volume']]

In [16]:
y=df[['fault_severity']]

In [17]:
x=pd.get_dummies(x,drop_first=True)

In [18]:
y=pd.get_dummies(y,drop_first=True)

In [19]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(x,y,test_size=0.2,random_state=42)

In [20]:
g=GaussianNB()
b=BernoulliNB()
k=KNeighborsClassifier()
log=LogisticRegression()
gbc=GradientBoostingClassifier()
r=RandomForestClassifier()
d=DecisionTreeClassifier()

In [21]:
algorithms=[g,b,k,log,gbc,r,d]
names=['GaussianNB','BernoulliNB','K Nearest','Logistic','GradientBoosting','RandomForest','DecisionTree'] 
def algo_test (A,b,algorithms=algorithms,names=names):
    # fit the data 
    for i in range(len(algorithms)):
        algorithms[i]=algorithms[i].fit(A,b)
    accuracy=[]
    precision=[]
    recall=[]
    f1=[]
    for i in range(len(algorithms)):
        accuracy.append(accuracy_score(y,algorithms[i].predict(x)),average='None')
        precision.append(precision_score(y,algorithms[i].predict(x)),average='None')
        recall.append(recall_score(y,algorithms[i].predict(x)),average='None')
        f1.append(f1_score(y,algorithms[i].predict(x)),average='None')
    metrics=pd.DataFrame(columns=['Accuracy','Precision','Recall','F1'],index=names)
    metrics['Accuracy']=accuracy 
    metrics['Precision']=precision 
    metrics['Recall']=recall 
    metrics['F1']=f1
    return metrics.sort_values('F1',ascending=False)  

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

In [ ]:
algo_test(x_train,y_train)